In [1]:
%pip install sagemaker

  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached importlib_metadata-6.11.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached schema-0.7.7-py2.py3-none-any.whl.metadata (34 kB)
  Using cached smdebug_rulesconfig-1.0.1-py2.py3-none-any.whl.metadata (943 bytes)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached antlr4-python3-runtime-4.9.3.tar.gz (117 kB)
  Preparing metadata (setup.py) ... done
  Using cached mock-4.0.3-py3-none-any.whl.metadata (2.8 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import yaml
import boto3

# import sagemaker
import sagemaker.session


from sagemaker.sklearn.estimator import SKLearn
from sagemaker.processing import (
    FrameworkProcessor,
    ProcessingInput,
    ProcessingOutput)
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.steps import ProcessingStep, CacheConfig
from sagemaker.workflow.functions import Join
from sagemaker.workflow.pipeline import Pipeline

/Users/julie.fisher/anaconda3/envs/explainibility_env/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/julie.fisher/Library/Application Support/sagemaker/config.yaml


In [3]:
session = sagemaker.session.Session()
pipe_session = PipelineSession()
region = session.boto_region_name
role = sagemaker.get_execution_role()
account_id = session.account_id()

tags = [
    {"Key": "Test", "Value": "me"}
   ]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 session = sagemaker.session.Session()                                                       │
│    2 pipe_session = PipelineSession()                                                            │
│    3 region = session.boto_region_name                                                           │
│    4 role = sagemaker.get_execution_role()                                                       │
│                                                                                                  │
│ /Users/julie.fisher/anaconda3/envs/explainibility_env/lib/python3.10/site-packages/sagemaker/ses │
│ sion.py:265 in __init__                                                                          │
│                                                                                                  │
│    262 │   │   self.lambda_client = None                                                         │
│    263 │   │   self.settings = settings if settings else SessionSettings()                       │
│    264 │   │                                                                                     │
│ ❱  265 │   │   self._initialize(                                                                 │
│    266 │   │   │   boto_session=boto_session,                                                    │
│    267 │   │   │   sagemaker_client=sagemaker_client,                                            │
│    268 │   │   │   sagemaker_runtime_client=sagemaker_runtime_client,                            │
│                                                                                                  │
│ /Users/julie.fisher/anaconda3/envs/explainibility_env/lib/python3.10/site-packages/sagemaker/ses │
│ sion.py:293 in _initialize                                                                       │
│                                                                                                  │
│    290 │   │                                                                                     │
│    291 │   │   self._region_name = self.boto_session.region_name                                 │
│    292 │   │   if self._region_name is None:                                                     │
│ ❱  293 │   │   │   raise ValueError(                                                             │
│    294 │   │   │   │   "Must setup local AWS configuration with a region supported by SageMaker  │
│    295 │   │   │   )                                                                             │
│    296                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Must setup local AWS configuration with a region supported by SageMaker.

In [ ]:
config_file = "configs/param_config_simp.yaml"
special_name = "debug"

In [ ]:
with open(f"{config_file}", "r",
          encoding="utf-8") as config_file:
    config_params = yaml.safe_load(config_file)

date_ymd = config_params["date_ts"][:10]
pipeline_name = config_params["use_case"] + "Prod"
model_package_group_name = config_params["use_case"].lower() + "Prod"
model_name = model_package_group_name + "-" + config_params["date_ts"]
prod_uri = "s3://{}".format(config_params["bucket"])
model_description = (config_params["target"].split("_")[0]
                     + config_params["filter_value"].capitalize()
                     + str(config_params["day_interval"])
                     + "Day"
                     + date_ymd.replace("-","")
                    )
if special_name is not None:
    model_description = model_description + special_name
process_input_path = "/opt/ml/processing/input"
process_output_path = "/opt/ml/processing/output"

source_dir_path = "src/source_dir/mobility-" + config_params[
    "date_ts"]

pipe_prefix = ["mobility_models", model_description]

data_filepath = ["s3://{}".format(config_params["bucket"]),
                 *pipe_prefix,
                 "data"]

step_counter = 1
step_list = []

step_cache_config = CacheConfig(
    enable_caching=True,
    expire_after=config_params["step_cache_config"])

In [ ]:
s3_resource = boto3.resource('s3')
s3_resource.Bucket(config_params['bucket']).upload_file('sql_queries/base_query.txt', f"s3://{config_params['bucket']}/{model_description}/sql_queries/base_query.txt")

In [ ]:
query_processor = FrameworkProcessor(
    estimator_cls=SKLearn,
    framework_version=config_params["framework_version"],
    role=role,
    instance_type=config_params["instance_type_processing"],
    instance_count=config_params["instance_count_processing"],
    sagemaker_session=pipe_session,
    base_job_name="data_query")

query_list = []

In [ ]:
# Query one table
# Set arguments
table = "WEB_DATA"
arguments = [
    "--table", table,
    "--input-path", process_input_path,
    "--output_path", process_output_path,
    "--db-type", "mysql",
    "--dbname", "machinelearning",
    "--folder-date", date_ymd,
    "--interval", config_params["day_interval"]
]

query_args = query_processor.run(
    code="feat_query.py",
    source_dir=source_dir_path,
    inputs=[
        ProcessingInput(
            source=Join(
                on="/",
                values=[
                    "s3://".format(config_params['bucket']),
                    model_description,
                    "sql_queries",
                    "base_query.txt"]),
            destination=process_input_path,
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="features",
            source=process_output_path,
            destination=Join(
                on="/",
                values=[
                    *data_filepath,
                    "features"])
        )
    ],
    arguments=arguments
)

query_dbs = ProcessingStep(
    name="{}-{}".format(step_counter, table),
    step_args=query_args,
    cache_config=step_cache_config
)
step_list.append(query_dbs)
query_list.append(query_dbs)

In [ ]:
# Create Pipeline

pipeline = Pipeline(
    name=pipeline_name,
    # parameters=param_list,
    steps=step_list,
    sagemaker_session=pipe_session)

with open("pipeline_definition_simp.json", "w", encoding="utf-8") as def_file:
    json.dump(json.loads(pipeline.definition()), def_file, indent=4)

# Upload Pipeline and Start Exedution

pipeline.upsert(role_arn=role, tags=tags)

pipeline.start(
    execution_display_name=model_description)